In [1]:
import numpy as np
import MDAnalysis as md
import traj_tools

In [12]:
# load trajectory data
topFile = "partially_unfolded_beta_sheet.xyz"
trajFile = "partially_unfolded_beta_sheet_v01.out.dcd"
coord = md.Universe(topFile,trajFile)
print("Number of atoms in molecule:", coord.atoms.n_atoms)
print("Number of frames in trajectory:", coord.trajectory.n_frames)

Number of atoms in molecule: 12
Number of frames in trajectory: 100000


In [13]:
# create array of trajectory position data and populate it
traj_positions = np.empty((coord.trajectory.n_frames,coord.atoms.n_atoms,3),dtype=np.float64)
for ts in coord.trajectory:
    traj_positions[ts.frame] = coord.atoms.positions

In [14]:
# uniform align and print trajectory
%time uniform_aligned_positions, uniform_aligned_avg, uniform_aligned_var = traj_tools.align_maximum_likelihood_uniform(traj_positions)
coord.atoms.positions = uniform_aligned_avg
coord.atoms.write("uniform_avg.pdb")
with md.Writer("uniform_aligned.dcd", coord.atoms.n_atoms) as W:
    for ts in coord.trajectory:
        coord.atoms.positions = uniform_aligned_positions[ts.frame]
        W.write(coord.atoms)

Iteration   Log Likelihood     Delta Log Likelihood
-----------------------------------------------------------------
2 2324111.316202661 5976403.882494385
3 3465342.6446245536 1141231.3284218926
4 3467478.339032037 2135.694407483563
5 3467489.260139128 10.921107091009617
6 3467489.3177907076 0.0576515793800354
7 3467489.3180954577 0.0003047501668334007
8 3467489.3180971025 1.644715666770935e-06
9 3467489.3180971146 1.210719347000122e-08
10 3467489.318097081 3.3527612686157227e-08
11 3467489.3180970587 2.2351741790771484e-08
12 3467489.318097065 6.51925802230835e-09
13 3467489.318097071 5.587935447692871e-09
14 3467489.31809707 9.313225746154785e-10
15 3467489.318097068 1.862645149230957e-09
16 3467489.318097068 0.0
CPU times: user 13.7 s, sys: 154 ms, total: 13.8 s
Wall time: 13.9 s


/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py

In [15]:
# intermediate align and print trajectory
%time intermediate_aligned_positions, intermediate_aligned_average, intermediate_aligned_vars = traj_tools.align_maximum_likelihood_intermediate(traj_positions)
coord.atoms.positions = intermediate_aligned_average
coord.add_TopologyAttr('tempfactors')
coord.atoms.tempfactors = intermediate_aligned_vars
coord.atoms.write("intermediate_avg.pdb")
with md.Writer("intermediate_aligned.dcd", coord.atoms.n_atoms) as W:
    for ts in coord.trajectory:
        coord.atoms.positions = intermediate_aligned_positions[ts.frame]
        W.write(coord.atoms)

Iteration   Log Likelihood     Delta Log Likelihood
-----------------------------------------------------------------
1 3681649.037414282 60864.15330319246
2 3711994.1208574055 30345.08344312338
3 3724293.6919626123 12299.571105206851
4 3732826.8876737524 8533.195711140055
5 3738935.074040105 6108.186366352718
6 3743185.642628816 4250.568588710856
7 3746052.604959839 2866.9623310230672
8 3747939.5328714447 1886.927911605686
9 3749160.6356755286 1221.1028040838428
10 3749942.4873912144 781.8517156857997
11 3750440.117002029 497.6296108146198
12 3750756.042947273 315.9259452437982
13 3750956.596019092 200.55307181924582
14 3751084.116073155 127.52005406282842
15 3751165.4207975 81.3047243449837
16 3751217.4354150174 52.01461751759052
17 3751250.835153086 33.399738068692386
18 3751272.3628900084 21.527736922260374
19 3751286.289629448 13.92673943983391
20 3751295.3304871786 9.040857730433345
21 3751301.2184817805 5.887994601856917
22 3751305.0644030115 3.845921230968088
23 3751307.5831511

In [17]:
# weighted align and print trajectory
%time weighted_aligned_positions, weighted_aligned_average, weighted_aligned_covar = traj_tools.align_maximum_likelihood_weighted(traj_positions,max_steps=500)
coord.atoms.positions = weighted_aligned_average
coord.add_TopologyAttr('tempfactors')
coord.atoms.tempfactors = np.diag(weighted_aligned_covar)
coord.atoms.write("weighted_avg.pdb")
with md.Writer("weighted_aligned.dcd", coord.atoms.n_atoms) as W:
    for ts in coord.trajectory:
        coord.atoms.positions = weighted_aligned_positions[ts.frame]
        W.write(coord.atoms)

Iteration   Log Likelihood     Delta Log Likelihood
-----------------------------------------------------------------
1 4241747.041597746 15147.711166127585
2 4251663.070285241 9916.028687494807
3 4260200.3971592635 8537.326874022372
4 4267948.426825261 7748.029665997252
5 4275054.425691726 7105.998866464943
6 4281594.93300614 6540.507314414717
7 4287626.439880849 6031.506874708459
8 4293196.073452206 5569.633571357466
9 4298344.939771422 5148.8663192158565
10 4303109.568688408 4764.628916985355
11 4307522.727496652 4413.158808244392
12 4311613.9710382745 4091.2435416225344
13 4315410.055869873 3796.084831598215
14 4318935.271189563 3525.2153196902946
15 4322211.712354228 3276.441164664924
16 4325259.511746018 3047.799391790293
17 4328097.036511809 2837.52476579044
18 4330741.0598675795 2644.023355770856
19 4333206.910932426 2465.851064846851
20 4335508.6069015255 2301.695969099179
21 4337658.970532025 2150.3636304996908
22 4339669.735289546 2010.7647575205192
23 4341551.640018037 1881

205 4374430.727383429 0.2351699536666274
206 4374430.951401306 0.22401787713170052
207 4374431.164795406 0.21339409984648228
208 4374431.368068902 0.20327349565923214
209 4374431.561701487 0.19363258499652147
210 4374431.7461497625 0.1844482757151127
211 4374431.921849045 0.17569928243756294
212 4374432.0892138705 0.16736482549458742
213 4374432.24863924 0.159425369463861
214 4374432.400501472 0.15186223201453686
215 4374432.545159098 0.14465762581676245
216 4374432.682953473 0.1377943754196167
217 4374432.814210062 0.13125658873468637
218 4374432.939238746 0.12502868380397558
219 4374433.058334822 0.11909607611596584
220 4374433.171779571 0.1134447492659092
221 4374433.2798409015 0.10806133039295673
222 4374433.3827741295 0.10293322801589966
223 4374433.480822416 0.09804828651249409
224 4374433.574217449 0.09339503291994333
225 4374433.663179876 0.0889624273404479
226 4374433.747919984 0.08474010787904263
227 4374433.828637902 0.08071791753172874
228 4374433.905524623 0.07688672095537

/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'resnames' Using default value of 'UNK'
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  warnings.warn("Found no information for attr: '{}'"
/Users/mmccull/opt/anaconda3/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py